In [0]:
%pip install mlflow>=3.1.3

In [0]:
%restart_python

In [0]:
import mlflow
from mlflow.client import MlflowClient

# Usa o tracking server "Databricks" do próprio workspace
mlflow.set_tracking_uri("databricks")
client = MlflowClient()

In [0]:
exps = client.search_experiments()
len(exps), [e.experiment_id for e in exps[:5]]

In [0]:
experiment_id = "2223811622375641"
exp = client.get_experiment(experiment_id)
(exp.experiment_id, exp.name)

In [0]:
import os, json, time, requests
import mlflow

mlflow.set_tracking_uri("databricks")

# 1) Descobrir o host do workspace: se você souber seu host, pode setar direto.
# Caso não saiba, preencha manualmente a variável `HOST`:
HOST = "https://adb-7534486785283121.1.azuredatabricks.net/"  # <-- ajuste para o seu workspace

# 2) Pegue o ID do experimento que você usa no app:
EXPERIMENT_ID = "2223811622375641"

# 3) Gere valores de teste:
now_ms = int(time.time() * 1000)
message_id = f"test-{now_ms}"

# 4) Corpo no schema esperado
payload = {
    "trace": {
        "info": {
            "experiment_id": EXPERIMENT_ID,
            "name": "chat_interaction",
            "client_request_id": message_id,
            "timestamp_ms": now_ms,
            "execution_time_ms": 120,
            "state": "OK",
            "tags": {"session": "notebook-check", "user": "tester"},
            "metadata": {"app": "chat-template", "feature": "trace-log"}
        },
        "data": {
            "request": json.dumps([{"role": "user", "content": "Olá"}]),
            "response": json.dumps([{"role": "assistant", "content": "Oi!"}]),
            "spans": []
        }
    }
}

# 5) Autenticação: dentro do workspace, o Databricks injeta credenciais;
#    o "requests" pode usar o header "Authorization: Bearer <token>".
#    Para não lidar com token manualmente aqui, o melhor é usar a lib 'requests' com 'databricks-sdk'
#    OU executar este POST via "dbutils.notebook" + utilities internas.
#    Como caminho mais portátil, peça um PAT temporário com a CLI (local) ou rode este teste como App.
#    >>> Se preferir, pule para a Opção B2 (search), que não precisa do POST.

url = f"{HOST.rstrip('/')}/api/3.0/mlflow/traces"

# Se você tiver um token do contexto (ex.: variável secreta/KeyVault), injete aqui:
TOKEN = os.environ.get("DATABRICKS_TOKEN")  # se existir no cluster/secret scope
headers = {"Authorization": f"Bearer {TOKEN}", "Content-Type": "application/json"}

resp = requests.post(url, headers=headers, data=json.dumps(payload), timeout=30)
print(resp.status_code, resp.text[:500])

In [0]:
# 0) Pré-requisitos
import json, time, requests
import mlflow

mlflow.set_tracking_uri("databricks")  # usa o tracking do workspace

# 1) Host + Token a partir do contexto do notebook (NÃO precisa saber o valor manualmente)
host  = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().getOrElse(None)
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)

print("Host:", host)
print("Token presente?", bool(token))  # não imprime o token, só confirma presença

# 2) Parâmetros do teste
EXPERIMENT_ID = "2223811622375641"   # o mesmo que você já usa no app
now_ms = int(time.time() * 1000)
message_id = f"test-{now_ms}"

# 3) Payload no schema esperado
payload = {
    "trace": {
        "info": {
            "experiment_id": EXPERIMENT_ID,
            "name": "chat_interaction",
            "client_request_id": message_id,
            "timestamp_ms": now_ms,
            "execution_time_ms": 120,
            "state": "OK",
            "tags": {"session": "notebook-check", "user": "tester"},
            "metadata": {"app": "chat-template", "feature": "trace-log"},
        },
        "data": {
            "request": json.dumps([{"role": "user", "content": "Olá"}]),
            "response": json.dumps([{"role": "assistant", "content": "Oi!"}]),
            "spans": [],
        },
    }
}

# 4) POST do trace
url = f"{host.rstrip('/')}/api/3.0/mlflow/traces"
headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}
resp = requests.post(url, headers=headers, json=payload, timeout=30)
print("POST status:", resp.status_code, resp.text[:500])

# 5) Buscar o trace por client_request_id (confirma ingestão)
search_url = f"{host.rstrip('/')}/api/3.0/mlflow/traces/search"
search_body = {
    "filter_string": f'trace.client_request_id = "{message_id}"',
    "max_results": 5
}
resp2 = requests.post(search_url, headers=headers, json=search_body, timeout=30)
print("SEARCH status:", resp2.status_code, resp2.text[:500])

In [0]:
# 0) Contexto + tracking
import json, time, requests
import mlflow
mlflow.set_tracking_uri("databricks")

host  = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().getOrElse(None)
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)
headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}

EXPERIMENT_ID = "2223811622375641"
now_ms = int(time.time() * 1000)
message_id = f"test-{now_ms}"

# 1) >>>>>> ALTERAÇÃO AQUI: trace_info / trace_data <<<<<<
payload = {
    "trace": {
        "trace_info": {
            "experiment_id": EXPERIMENT_ID,
            "name": "chat_interaction",
            "client_request_id": message_id,
            "timestamp_ms": now_ms,
            "execution_time_ms": 120,
            "state": "OK",
            "tags": {"session": "notebook-check", "user": "tester"},
            "metadata": {"app": "chat-template", "feature": "trace-log"}
        },
        "trace_data": {
            "request": json.dumps([{"role": "user", "content": "Olá"}]),
            "response": json.dumps([{"role": "assistant", "content": "Oi!"}]),
            "spans": []
        }
    }
}

url = f"{host.rstrip('/')}/api/3.0/mlflow/traces"
resp = requests.post(url, headers=headers, json=payload, timeout=30)
print("POST status:", resp.status_code, resp.text[:500])


In [0]:
# 0) Contexto + tracking (iguais)
import json, time, requests
import mlflow
mlflow.set_tracking_uri("databricks")

host  = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().getOrElse(None)
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)
headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}

EXPERIMENT_ID = "2223811622375641"
now_ms = int(time.time() * 1000)
message_id = f"test-{now_ms}"

# 1) >>>> payload com trace_info.trace_location <<<<
payload = {
    "trace": {
        "trace_info": {
            "experiment_id": EXPERIMENT_ID,     # ainda é útil para leitura/atalhos
            "trace_location": {                 # <-- CAMPO NOVO OBRIGATÓRIO
                "mlflow_experiment": {
                    "experiment_id": EXPERIMENT_ID
                }
            },
            "name": "chat_interaction",
            "client_request_id": message_id,
            "timestamp_ms": now_ms,
            "execution_time_ms": 120,
            "state": "OK",
            "tags": {"session": "notebook-check", "user": "tester"},
            "metadata": {"app": "chat-template", "feature": "trace-log"},
        },
        "trace_data": {
            "request": json.dumps([{"role": "user", "content": "Olá"}]),
            "response": json.dumps([{"role": "assistant", "content": "Oi!"}]),
            "spans": []
        }
    }
}

# 2) POST /traces (v3)
url = f"{host.rstrip('/')}/api/3.0/mlflow/traces"
resp = requests.post(url, headers=headers, json=payload, timeout=30)
print("POST status:", resp.status_code, resp.text[:500])

# 3) SEARCH /traces/search com locations (exige localização)
search_url = f"{host.rstrip('/')}/api/3.0/mlflow/traces/search"
search_body = {
    "filter_string": f'trace.client_request_id = "{message_id}"',
    "locations": [EXPERIMENT_ID],  # lista de experimentos para buscar
    "max_results": 5
}
resp2 = requests.post(search_url, headers=headers, json=search_body, timeout=30)
print("SEARCH status:", resp2.status_code, resp2.text[:800])

In [0]:
# 0) Contexto + tracking (iguais)
import json, time, requests
import mlflow
mlflow.set_tracking_uri("databricks")

host  = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().getOrElse(None)
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)
headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}

EXPERIMENT_ID = "2223811622375641"
now_ms = int(time.time() * 1000)
message_id = f"test-{now_ms}"

# 1) >>> payload com trace_info.trace_location E trace_info.request_time <<<
payload = {
    "trace": {
        "trace_info": {
            # localização do trace: experimento do MLflow
            "trace_location": {
                "mlflow_experiment": {
                    "experiment_id": EXPERIMENT_ID
                }
            },

            # canônicos em V3:
            "request_time": now_ms,          # <<-- OBRIGATÓRIO (início, em ms)
            "execution_time_ms": 120,
            "state": "OK",

            # identificadores/opcionais
            "experiment_id": EXPERIMENT_ID,  # atalho conveniente (lido pela UI/SDK)
            "name": "chat_interaction",
            "client_request_id": message_id,

            # contexto pesquisável
            "tags": {"session": "notebook-check", "user": "tester"},
            "metadata": {"app": "chat-template", "feature": "trace-log"},
        },
        "trace_data": {
            "request": json.dumps([{"role": "user", "content": "Olá"}]),
            "response": json.dumps([{"role": "assistant", "content": "Oi!"}]),
            "spans": []
        }
    }
}

# 2) POST /api/3.0/mlflow/traces  (família 3.0 é a mesma do search)
url = f"{host.rstrip('/')}/api/3.0/mlflow/traces"
resp = requests.post(url, headers=headers, json=payload, timeout=30)
print("POST status:", resp.status_code, resp.text[:500])

# 3) SEARCH /api/3.0/mlflow/traces/search  (forneça 'locations' como OBJETOS)
search_url = f"{host.rstrip('/')}/api/3.0/mlflow/traces/search"
search_body = {
    "filter_string": f'trace.client_request_id = "{message_id}"',
    "locations": [                       # <<-- precisa ser lista de OBJETOS
        {"mlflow_experiment": {"experiment_id": EXPERIMENT_ID}}
    ],
    "max_results": 5
}
resp2 = requests.post(search_url, headers=headers, json=search_body, timeout=30)
print("SEARCH status:", resp2.status_code, resp2.text[:800])

In [0]:
# === TESTE ÚNICO: request_time ISO + execution_duration como string "Xs" ===
import json, time, requests
from datetime import datetime, timezone
import mlflow
mlflow.set_tracking_uri("databricks")

host  = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().getOrElse(None)
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)
headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}

EXPERIMENT_ID = "2223811622375641"
now_ms = int(time.time() * 1000)
message_id = f"test-rt-iso-durstr-{now_ms}"

# 1) request_time em ISO‑8601 UTC com milissegundos + 'Z'
request_time_iso = datetime.fromtimestamp(now_ms / 1000, tz=timezone.utc) \
                           .isoformat(timespec="milliseconds") \
                           .replace("+00:00", "Z")

# 2) Duração de 120 ms → "0.120s"
execution_duration_str = "0.120s"

payload = {
    "trace": {
        "trace_info": {
            "trace_location": {
                "mlflow_experiment": {"experiment_id": EXPERIMENT_ID}
            },
            "request_time": request_time_iso,        # Timestamp (RFC 3339)
            "execution_duration": execution_duration_str,  # Duration (string "Xs")
            "state": "OK",
            "experiment_id": EXPERIMENT_ID,
            "name": "chat_interaction",
            "client_request_id": message_id,
            "tags": {"session": "notebook-check", "user": "tester"},
            "metadata": {"app": "chat-template", "feature": "trace-log"},
        },
        "trace_data": {
            "request": json.dumps([{"role": "user", "content": "Olá"}]),
            "response": json.dumps([{"role": "assistant", "content": "Oi!"}]),
            "spans": []
        }
    }
}

url = f"{host.rstrip('/')}/api/3.0/mlflow/traces"
resp = requests.post(url, headers=headers, json=payload, timeout=30)
print("POST (request_time ISO + execution_duration 'Xs') status:", resp.status_code, resp.text[:3000])

In [0]:
# === TESTE ÚNICO: usar trace_info.trace_metadata com as chaves reservadas ===
import json, time, requests
from datetime import datetime, timezone
import mlflow
mlflow.set_tracking_uri("databricks")

host  = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().getOrElse(None)
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)
headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}

EXPERIMENT_ID = "2223811622375641"
now_ms = int(time.time() * 1000)
message_id = f"test-tracemeta-session-{now_ms}"

# Campos já validados:
request_time_iso = datetime.fromtimestamp(now_ms / 1000, tz=timezone.utc) \
                           .isoformat(timespec="milliseconds") \
                           .replace("+00:00", "Z")
execution_duration_str = "0.120s"

payload = {
    "trace": {
        "trace_info": {
            "trace_location": {
                "mlflow_experiment": {"experiment_id": EXPERIMENT_ID}
            },
            "request_time": request_time_iso,             # RFC 3339
            "execution_duration": execution_duration_str, # "Xs"
            "state": "OK",

            "experiment_id": EXPERIMENT_ID,
            "name": "chat_interaction",
            "client_request_id": message_id,

            # >>> AQUI: use o campo correto 'trace_metadata' com as chaves padronizadas <<<
            "trace_metadata": {
                "mlflow.trace.session": "sessao-oficial-002",
                "mlflow.trace.user":    "wilker.paz@empresa"
            },

            # (Opcional) pode manter 'tags' para outros filtros, mas não são usados para Session/User
            # "tags": { "qualquer_tag": "valor" },
        },
        "trace_data": {
            "request": json.dumps([{"role": "user", "content": "Olá"}]),
            "response": json.dumps([{"role": "assistant", "content": "Oi!"}]),
            "spans": []
        }
    }
}

url = f"{host.rstrip('/')}/api/3.0/mlflow/traces"
resp = requests.post(url, headers=headers, json=payload, timeout=30)
print("POST (trace_metadata session/user) status:", resp.status_code, resp.text[:300])